<a href="https://colab.research.google.com/github/Cutie-tee/Roboreviews_project/blob/main/reviews_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset consists of 3 files: 1429_1.csv
Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv
Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv




In [2]:
!pip install --upgrade pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 109.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [3]:
import pandas as pd

def safe_read_csv(file_path):
    """
    Safely reads a CSV file by handling parsing issues.
    """
    try:
        # Read the CSV with specific options for handling errors
        return pd.read_csv(
            file_path,
            on_bad_lines='skip',  # Skips problematic lines
            quotechar='"',       # Specifies the quote character
            escapechar='\\',     # Escapes special characters
            engine='python'      # Use Python engine for better handling of malformed rows
        )
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None

# Load datasets safely
file1_data = safe_read_csv('1429_1.csv')
file2_data = safe_read_csv('Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv')
file3_data = safe_read_csv('Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')

# Check if all files loaded successfully
if file1_data is not None and file2_data is not None and file3_data is not None:
    # Standardizing column names
    file1_data.rename(columns=lambda x: x.strip(), inplace=True)
    file2_data.rename(columns=lambda x: x.strip(), inplace=True)
    file3_data.rename(columns=lambda x: x.strip(), inplace=True)

    # Align datasets to common columns
    common_columns = list(set(file1_data.columns) & set(file2_data.columns) & set(file3_data.columns))

    # Selecting only common columns
    file1_data = file1_data[common_columns]
    file2_data = file2_data[common_columns]
    file3_data = file3_data[common_columns]

    # Concatenate datasets
    combined_data = pd.concat([file1_data, file2_data, file3_data], ignore_index=True)

    # Dropping duplicates
    combined_data.drop_duplicates(inplace=True)

    # Resetting index
    combined_data.reset_index(drop=True, inplace=True)

    # Save cleaned dataset
    combined_data.to_csv('combined_reviews_cleaned.csv', index=False)

    # Display overview
    print("Dataset successfully cleaned and saved.")
    print(combined_data.info())
else:
    print("One or more files could not be loaded.")


Dataset successfully cleaned and saved.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7278 entries, 0 to 7277
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   7278 non-null   object 
 1   reviews.username     7275 non-null   object 
 2   reviews.date         7268 non-null   object 
 3   reviews.rating       7254 non-null   float64
 4   reviews.sourceURLs   7277 non-null   object 
 5   reviews.id           8 non-null      float64
 6   categories           7277 non-null   object 
 7   reviews.numHelpful   4418 non-null   float64
 8   asins                7277 non-null   object 
 9   name                 7277 non-null   object 
 10  brand                7277 non-null   object 
 11  reviews.text         7276 non-null   object 
 12  reviews.title        7276 non-null   object 
 13  manufacturer         7277 non-null   object 
 14  reviews.dateSeen     7277 non-null   object 
 15

In [4]:
# Load the combined dataset
import pandas as pd

# Load the cleaned dataset
combined_data = pd.read_csv('combined_reviews_cleaned.csv')

# Display initial missing values
print("Missing Values Before Processing:")
print(combined_data.isnull().sum())

# 1. Drop rows with missing `reviews.text` (essential for all tasks)
combined_data = combined_data.dropna(subset=['reviews.text'])

# 2. Drop rows with missing `reviews.rating` (essential for classification and clustering)
combined_data = combined_data.dropna(subset=['reviews.rating'])

# 3. Optional: Handle missing values in other columns (example: reviews.doRecommend)
# Here, replace missing values with a default or mode value
combined_data['reviews.doRecommend'] = combined_data['reviews.doRecommend'].fillna('Unknown')

# 4. Drop columns with minimal or irrelevant data (e.g., `reviews.id` with only 71 non-null values)
columns_to_drop = ['reviews.id']
combined_data = combined_data.drop(columns=columns_to_drop, errors='ignore')

# Display missing values after processing
print("\nMissing Values After Processing:")
print(combined_data.isnull().sum())

# Save the preprocessed dataset
preprocessed_dataset_path = 'preprocessed_reviews.csv'
combined_data.to_csv(preprocessed_dataset_path, index=False)

print(f"Preprocessed dataset saved to {preprocessed_dataset_path}.")



Missing Values Before Processing:
id                        0
reviews.username          3
reviews.date             10
reviews.rating           24
reviews.sourceURLs        1
reviews.id             7270
categories                1
reviews.numHelpful     2860
asins                     1
name                      1
brand                     1
reviews.text              2
reviews.title             2
manufacturer              1
reviews.dateSeen          1
keys                      1
reviews.doRecommend    2868
dtype: int64

Missing Values After Processing:
id                        0
reviews.username          2
reviews.date              1
reviews.rating            0
reviews.sourceURLs        0
categories                0
reviews.numHelpful     2835
asins                     0
name                      0
brand                     0
reviews.text              0
reviews.title             1
manufacturer              0
reviews.dateSeen          0
keys                      0
reviews.doRecommend    

In [5]:
combined_data['reviews.username'] = combined_data['reviews.username'].fillna("Anonymous")
combined_data['name'] = combined_data['name'].fillna("Unknown")

combined_data['reviews.doRecommend'] = combined_data['reviews.doRecommend'].map({'Yes': True, 'No': False})

assert combined_data['reviews.rating'].between(1, 5).all()

combined_data.to_csv('final_preprocessed_reviews.csv', index=False)
print("Final preprocessed dataset saved.")



Final preprocessed dataset saved.


**Sentinent analysis  with RoBerta-base**

In [6]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 2.2.3 which is incompatible.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [7]:
import pandas as pd

# Load the dataset
data = pd.read_csv('final_preprocessed_reviews.csv', low_memory=False)

# Get the number of rows
num_rows = data.shape[0]

print(f"Number of rows in the dataset: {num_rows}")


Number of rows in the dataset: 7253


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
from datasets import Dataset
import torch

# Disable Weights & Biases (W&B) logging
import os
os.environ["WANDB_DISABLED"] = "true"

# Load the final_preprocessed dataset
data = pd.read_csv('final_preprocessed_reviews.csv', low_memory=False)

# Map ratings to sentiment using the given labels
def map_rating_to_sentiment(rating):
    if rating > 3:
        return 2  # Positive
    elif rating == 3:
        return 1  # Neutral
    else:
        return 0  # Negative

data['sentiment'] = data['reviews.rating'].apply(map_rating_to_sentiment)

# Handle class imbalance with oversampling
positive_data = data[data['sentiment'] == 2]
neutral_data = data[data['sentiment'] == 1]
negative_data = data[data['sentiment'] == 0]

# Oversample minority classes to match the Positive class size
neutral_oversampled = resample(neutral_data, replace=True, n_samples=len(positive_data), random_state=42)
negative_oversampled = resample(negative_data, replace=True, n_samples=len(positive_data), random_state=42)

# Combine back into one dataset
balanced_data = pd.concat([positive_data, neutral_oversampled, negative_oversampled])

# Split the balanced data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    balanced_data['reviews.text'], balanced_data['sentiment'], test_size=0.2, random_state=42
)

# Convert to Hugging Face Dataset format
train_df = pd.DataFrame({"text": train_texts, "label": train_labels})
test_df = pd.DataFrame({"text": test_texts, "label": test_labels})

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenizer and model setup
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 classes

# Tokenize datasets
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

# Compute class weights for weighted loss
class_weights = torch.tensor(
    [1.0 / len(negative_data), 1.0 / len(neutral_data), 1.0 / len(positive_data)], dtype=torch.float
).to("cuda" if torch.cuda.is_available() else "cpu")

# Modify the model's loss function to include class weights
from torch.nn import CrossEntropyLoss
model.config.problem_type = "single_label_classification"
model.classifier.weight = class_weights

# Training arguments with epoch-based evaluation, early stopping, and adjusted learning rate
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Evaluate once per epoch
    save_strategy="epoch",        # Save model once per epoch
    learning_rate=1e-5,           # Lower learning rate for better convergence
    per_device_train_batch_size=8,
    num_train_epochs=10,          # Set high epochs; early stopping will stop training earlier
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=2,
    report_to="none",             # Disable W&B and other integrations
    load_best_model_at_end=True,  # Load the best model at the end
    metric_for_best_model="eval_loss",  # Monitor validation loss
    greater_is_better=False       # Smaller loss is better
)

# Define evaluation metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="macro")
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer with EarlyStoppingCallback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stop if no improvement for 2 evaluations
)

# Train the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_roberta_sentiment")
tokenizer.save_pretrained("./fine_tuned_roberta_sentiment")

print("Model fine-tuned and saved successfully!")




tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Map:   0%|          | 0/15976 [00:00<?, ? examples/s]

Map:   0%|          | 0/3995 [00:00<?, ? examples/s]

<ipython-input-8-275cee15637f>:107: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.174400,0.076961,0.985982,0.986467,0.985965,0.985980
2,0.121000,0.099703,0.986733,0.987281,0.986717,0.986774
3,0.002400,0.084605,0.988235,0.988708,0.988221,0.988284


Model fine-tuned and saved successfully!


In [9]:
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Get predictions on the test dataset
predictions = trainer.predict(test_dataset)

# Extract logits and true labels
logits = predictions.predictions
true_labels = predictions.label_ids

# Convert logits to predicted labels
predicted_labels = np.argmax(logits, axis=1)

# Evaluate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print("Accuracy:", accuracy)

# Generate a classification report
print("Classification Report:")
print(classification_report(true_labels, predicted_labels, target_names=["Negative", "Neutral", "Positive"]))


Accuracy: 0.9859824780976221
Classification Report:
              precision    recall  f1-score   support

    Negative       0.99      1.00      0.99      1307
     Neutral       0.97      1.00      0.98      1358
    Positive       1.00      0.96      0.98      1330

    accuracy                           0.99      3995
   macro avg       0.99      0.99      0.99      3995
weighted avg       0.99      0.99      0.99      3995



In [11]:
#Predict sentiment for new reviews using the fine-tuned model:

from transformers import pipeline

# Load the fine-tuned model
sentiment_classifier = pipeline(
    "text-classification",
    model="./fine_tuned_roberta_sentiment",
    tokenizer="./fine_tuned_roberta_sentiment"
)

# Predict sentiment for a new review
new_review = "This product is amazing! It exceeded all my expectations."
prediction = sentiment_classifier(new_review)
print(prediction)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'LABEL_2', 'score': 0.9990708827972412}]


To build Build the Article-Generating Model

Testing the model